In [ ]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use(['science', 'no-latex'])  # Optional: replace 'no-latex' with 'ieee' if using LaTeX

In [ ]:
# Get dataframe from csv named 'parity_data.csv'
df = pd.read_csv('parity_data.csv')

In [ ]:
df.head()  # Display the first few rows of the dataframe

In [ ]:
# display the column names of df
print(df.columns)

In [ ]:
df = df[[col for col in df.columns if 'MIN' not in col and 'MAX' not in col and 'step' not in col]]
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use(['science', 'ieee'])  # Enables LaTeX + clean scientific styling

# LaTeX font settings
plt.rcParams.update({
    'text.usetex': True,
    'font.family': 'serif',
    'font.size': 8,
    'axes.titlesize': 9,
    'axes.labelsize': 8,
    'legend.fontsize': 7,
    'xtick.labelsize': 7,
    'ytick.labelsize': 7
})

# Filter columns
filtered_df = df[[col for col in df.columns if all(x not in col for x in ['MIN', 'MAX', 'step'])]]

methods = ['prefix-sum', 'maj-voting', 'coa']
sequence_lengths = df['sequence_length']
best_accs = {method: [] for method in methods}
error_bars = {method: [] for method in methods}

for method in methods:
    method_cols = [col for col in filtered_df.columns if method in col and 'avg_accuracy' in col]
    for _, row in df.iterrows():
        # Find best accuracy for this method at this sequence length
        method_accs = [row[col] for col in method_cols if pd.notna(row[col])]
        
        if method_accs:
            best_acc = max(method_accs)
            best_accs[method].append(best_acc)
            
            # Calculate standard error: sqrt(p * (1-p) / n) where n=100 runs
            std_error = np.sqrt(best_acc * (1 - best_acc) / 100)
            error_bars[method].append(std_error)
        else:
            best_accs[method].append(0)
            error_bars[method].append(0)

# Plotting
bar_width = 0.25
x_pos = range(len(sequence_lengths))

fig, ax = plt.subplots(figsize=(3.5, 2.2))  # Paper-friendly size
colors = ['#4C72B0', '#55A868', '#C44E52']

for i, method in enumerate(methods):
    ax.bar(
        [p + i * bar_width for p in x_pos],
        best_accs[method],
        width=bar_width,
        label=method.replace('-', ' ').title(),
        color=colors[i],
        yerr=error_bars[method],
        capsize=2,
        error_kw={'linewidth': 0.5, 'capthick': 0.5}
    )

ax.set_xticks([p + bar_width for p in x_pos])
ax.set_xticklabels(sequence_lengths)
ax.set_xlabel(r'\textbf{Sequence Length}')
ax.set_ylabel(r'\textbf{Accuracy}')
ax.legend(frameon=False, loc='upper right')
ax.grid(True, axis='y', linestyle='--', linewidth=0.5, alpha=0.7)

fig.tight_layout()
fig.savefig("accuracy_vs_sequence_length.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# Extract best hyperparameter values for each agent type and sequence length
import re

def extract_hyperparams(col_name):
    """Extract hyperparameters from column name"""
    if 'maj-voting' in col_name:
        match = re.search(r'agents(\d+)', col_name)
        return int(match.group(1)) if match else None
    elif 'coa' in col_name:
        match = re.search(r'chunk(\d+)', col_name)
        return int(match.group(1)) if match else None
    elif 'prefix-sum' in col_name:
        match = re.search(r'b(\d+)', col_name)
        return int(match.group(1)) if match else None
    return None

# Create results dictionary
best_hyperparams = {
    'maj-voting': {},
    'coa': {},  
    'prefix-sum': {}
}

for method in ['maj-voting', 'coa', 'prefix-sum']:
    method_cols = [col for col in filtered_df.columns if method in col and 'avg_accuracy' in col]
    
    for seq_len in sequence_lengths:
        row_data = filtered_df[filtered_df['sequence_length'] == seq_len].iloc[0]
        
        # Get accuracies for this method and sequence length
        method_accs = {}
        for col in method_cols:
            acc = row_data[col]
            if pd.notna(acc):  # Only include non-NaN values
                hyperparam = extract_hyperparams(col)
                if hyperparam is not None:
                    method_accs[hyperparam] = acc
        
        # Find best hyperparameter
        if method_accs:
            best_hyperparam = max(method_accs.keys(), key=lambda k: method_accs[k])
            best_hyperparams[method][seq_len] = {
                'hyperparam': best_hyperparam,
                'accuracy': method_accs[best_hyperparam]
            }

# Display results
print("Best hyperparameters for each agent type and sequence length:")
print("=" * 60)

for method in ['maj-voting', 'coa', 'prefix-sum']:
    print(f"\n{method.upper().replace('-', ' ')}:")
    print("-" * 40)
    
    if method == 'maj-voting':
        param_name = "num_agents"
    elif method == 'coa':
        param_name = "chunk_size"
    else:  # prefix-sum
        param_name = "branching_factor"
    
    for seq_len in sorted(best_hyperparams[method].keys()):
        result = best_hyperparams[method][seq_len]
        print(f"Seq length {seq_len:3d}: {param_name}={result['hyperparam']:2d}, accuracy={result['accuracy']:.3f}")

# Create a summary table
summary_data = []
for method in ['maj-voting', 'coa', 'prefix-sum']:
    for seq_len in sorted(best_hyperparams[method].keys()):
        result = best_hyperparams[method][seq_len]
        summary_data.append({
            'Method': method.replace('-', '_'),
            'Sequence Length': seq_len,
            'Best Hyperparam': result['hyperparam'],
            'Best Accuracy': result['accuracy']
        })

summary_df = pd.DataFrame(summary_data)
print(f"\n\nSummary Table:")
print("=" * 60)
print(summary_df.to_string(index=False))